In [1]:
import pandas as pd 
import os
import torch
from torch.utils.data import DataLoader, random_split
from torch import nn as nn
import  torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import  transforms
from torchvision.transforms import functional
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
from torchmetrics import Recall, Precision, Accuracy, AUROC

/var/folders/kf/yv1zrfvd701c47vqp5xz4sjh0000gn/T/ipykernel_3837/3624518383.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
img = Image.open('data/train_images/6103.jpg')
num_channels = functional.get_image_num_channels(img)
print(num_channels)

3


In [3]:
class ImageDataset: 
    def __init__(self, csv_path, image_dir, transform):
        self.data = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform
        self.data = self.data[self.data['image_id'].apply(lambda x: os.path.exists(os.path.join(self.image_dir, x)))]
        # self.data = self.data.map({"0": "Cassava Bacterial Blight (CBB)", 
        #                         "1": "Cassava Brown Streak Disease (CBSD)", 
        #                         "2": "Cassava Green Mottle (CGM)", 
        #                         "3": "Cassava Mosaic Disease (CMD)", 
        #                         "4": "Healthy"})


    def __len__(self): 
        return len(self.data)
    
    def __getitem__(self, idx): 
        image_name = os.path.join(self.image_dir, self.data.iloc[idx, 0])
  
        
        label = self.data.iloc[idx, 1]

        image = Image.open(image_name)
        
        if self.transform:
            image = self.transform(image)
    
        return image, label 

In [4]:
transforma = transforms.Compose([
    transforms.Resize((64, 64)),        # Resize images to 128x128
    transforms.ToTensor(),                # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize with mean & std
])

In [5]:
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
transform = weights.transforms()

In [6]:
img_dir = 'data/train_images'
csv_dir = 'data/train.csv'

In [7]:
files = ImageDataset(csv_path=csv_dir, image_dir=img_dir, transform=transforma)

In [8]:
train_size = int(0.8 * len(files))
test_size = len(files) - train_size

train_dataset, test_dataset = random_split(files, [train_size, test_size])

In [9]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)


In [10]:
# for x, y in train_loader: 
#    print(y)

In [11]:
crtiterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

In [12]:
# acc = Accuracy(task='multiclass', num_classes=5, average= 'weighted')
# pre = Precision(task='multiclass', num_classes=5, average='weighted')
# recall = Recall(task='multiclass', num_classes=5, average='weighted')

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(4): 
    model.train()
    running_loss = 0.0
    for feature, label in train_dataset:
        # feature, label = feature.to(device), label.to(device)
        optimizer.zero_grad()
        pred = model(feature.unsqueeze(0)).squeeze(0)
        loss = crtiterion(pred, torch.tensor(label))
        loss.backward()
        optimizer.step()
    running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{4}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/4], Loss: 0.0013
Epoch [2/4], Loss: 0.0006
Epoch [3/4], Loss: 0.0002
Epoch [4/4], Loss: 0.0002
